In [1]:
import torch
from transformers import AutoModelForCausalLM

from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl2.utils.io import load_pil_images

Python version is above 3.10, patching the collections module.


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.6.0+cu124)
    Python  3.10.13 (you have 3.10.16)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
/blue/changzhao/haoyuliao/.conda/envs/hao-vl2/lib/python3.10/site-packages/xformers/ops/swiglu_op.py:107: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(cls, ctx, x, w1, b1, w2, b2, w3, b3):
/blue/changzhao/haoyuliao/.conda/envs/hao-vl2/lib/python3.10/site-packages/xformers/ops/swiglu_op.py:128: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(cls, ctx, dx5):


<h1>Load model<h1/>

In [ ]:
# specify the path to the model
model_path = "deepseek-ai/deepseek-vl2" #"deepseek-ai/deepseek-vl2-tiny"
vl_chat_processor: DeepseekVLV2Processor = DeepseekVLV2Processor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

/home/haoyuliao/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Add pad token = ['<｜▁pad▁｜>'] to the tokenizer
<｜▁pad▁｜>:2
Add image token = ['<image>'] to the tokenizer
<image>:128815
Add grounding-related tokens = ['<|ref|>', '<|/ref|>', '<|det|>', '<|/det|>', '<|grounding|>'] to the tokenizer with input_ids
<|ref|>:128816
<|/ref|>:128817
<|det|>:128818
<|/det|>:128819
<|grounding|>:128820
Add chat tokens = ['<|User|>', '<|Assistant|>'] to the tokenizer with input_ids
<|User|>:128821
<|Assistant|>:128822



Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

<h1>Mixture-of-Experts Vision-Language Models for Advanced Multimodal Understanding <h1/>

In [24]:
## single image conversation example
## Please note that <|ref|> and <|/ref|> are designed specifically for the object localization feature. These special tokens are not required for normal conversations.
## If you would like to experience the grounded captioning functionality (responses that include both object localization and reasoning), you need to add the special token <|grounding|> at the beginning of the prompt. Examples could be found in Figure 9 of our paper.


question = "Can you tell me what are in the images?"
#question = "Do you see people is hiking?" #Now, dertermine which CES. Cultural ecosystem services (CES) are intangible, non-material benefits derived from nature through direct human outdoor interactions with the natural environment. In this study, we are interested in detecting 12 CES classes: Hiking, Biking, Boating, Camping, Fishing, Horseback Riding, Hunting, Shelling, Surfing, Swimming, Wildlife Viewing, Landscape Aesthetics. We define each class as follow. Hiking: the activity of going for long walks, especially in the country or woods. Biking: the activity of riding a bicycle. Boating: rowing or sailing in boats as a sport or form of recreation. Camping: the activity of spending a vacation living in a camp, tent, or camper. Fishing: the activity of catching fish. Horse riding: the sport or activity of riding horses. Hunting: the activity of hunting wild animals. Shelling: activity of collecting seashells. Surfing: the sport or pastime of riding a wave towards the shore while standing or lying on a surfboard. Swimming: swimming. Wildlife viewing: the act of observing wild animals and plants in their natural environment. Landscape aesthetics: the enjoyment and pleasure felt through the observation of environmental scenery. Now determine if an image contains one of the above 12 CES activities. If yes, tell me which class it belongs to. Only give me short answers, no need to provide the reasoning."
#question = "Cultural ecosystem services (CES) are intangible, non-material benefits derived from nature through direct human outdoor interactions with the natural environment. In this study, we are interested in detecting 12 CES classes: Hiking, Biking, Boating, Camping, Fishing, Horseback Riding, Hunting, Shelling, Surfing, Swimming, Wildlife Viewing, Landscape Aesthetics. We define each class as follow. Hiking: the activity of going for long walks, especially in the country or woods. Biking: the activity of riding a bicycle. Boating: rowing or sailing in boats as a sport or form of recreation. Camping: the activity of spending a vacation living in a camp, tent, or camper. Fishing: the activity of catching fish. Horse riding: the sport or activity of riding horses. Hunting: the activity of hunting wild animals. Shelling: activity of collecting seashells. Surfing: the sport or pastime of riding a wave towards the shore while standing or lying on a surfboard. Swimming: swimming. Wildlife viewing: the act of observing wild animals and plants in their natural environment. Landscape aesthetics: the enjoyment and pleasure felt through the observation of environmental scenery. Now determine if an image contains one of the above 12 CES activities. If yes, tell me which class it belongs to. Only give me short answers, no need to provide the reasoning."
#question = "Cultural ecosystem services (CES) are intangible, non-material benefits derived from nature through direct human outdoor interactions with the natural environment. In this study, we are interested in detecting 12 CES classes: Hiking, Biking, Boating, Camping, Fishing, Horseback Riding, Hunting, Shelling, Surfing, Swimming, Wildlife Viewing, Landscape Aesthetics. We define each class as follow. Hiking: the activity of going for long walks, especially in the country or woods. Biking: the activity of riding a bicycle. Boating: rowing or sailing in boats as a sport or form of recreation. Camping: the activity of spending a vacation living in a camp, tent, or camper. Fishing: the activity of catching fish. Horse riding: the sport or activity of riding horses. Hunting: the activity of hunting wild animals. Shelling: activity of collecting seashells. Surfing: the sport or pastime of riding a wave towards the shore while standing or lying on a surfboard. Swimming: swimming. Wildlife viewing: the act of observing wild animals and plants in their natural environment. Landscape aesthetics: the enjoyment and pleasure felt through the observation of environmental scenery. Now determine if an image contains one of the above 12 CES activities. If yes, tell me which class it belongs to. If not belog 12 CES activities, reply No CES. Only give me short answers for which class, no need to provide the reasoning."
#question = "Do you see Landscape Aesthetics (the enjoyment and pleasure felt through the observation of environmental scenery)?"
#Hiking, Biking, Boating, Camping, Fishing, Horseback Riding, Hunting, Shelling, Surfing, Swimming, Wildlife Viewing, Landscape Aesthetics?"

#question = "Do you see people is biking defiend this is a person riding a bicycle, a bike, or biking equipment off-road and over rough terrain and mountain trails?"

#image_path = "./832_49357079873_actual_Boating.png"
#image_path = "./5241_27602172055_actual_No_CES.png"
#image_path = "./CES_Images//5024_48100184206_actual_No_CES.png"
image_path1 = "./CES_Images//5059_35451390872_actual_No_CES.png"
image_path2 = "./CES_Images//832_49357079873_actual_Boating.png"
image_path3 = "./CES_Images//5051_28946084657_actual_No_CES.png"
question = "Describe the content of the image"#"How many people in the image?"#"Can you tell me what are in the images?"

image_path = "./CES_Images//832_49357079873_actual_Boating.png"

conversation = [
    {
        "role": "<|User|>",
        "content": f"{question}",
        "images": [f"{image_path}"],
    },
    {"role": "<|Assistant|>", "content": ""},
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=pil_images,
    force_batchify=True,
    system_prompt=""
).to(vl_gpt.device)

# run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# run the model to get the response
outputs = vl_gpt.language.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=False)
print(f"{prepare_inputs['sft_format'][0]}", answer)

<|User|>: Describe the content of the image

<|Assistant|>: The image appears to be a digital representation of a book cover. It features a central illustration of a person, possibly a woman, with long hair, wearing a dress. The background is a gradient of colors, transitioning from a darker shade at the top to a lighter one at the bottom. There are no visible texts or additional elements on the cover.<｜end▁of▁sentence｜>


In [20]:
#question = "Describe the image."
#question = "Do you see people is hiking?" #Now, dertermine which CES. Cultural ecosystem services (CES) are intangible, non-material benefits derived from nature through direct human outdoor interactions with the natural environment. In this study, we are interested in detecting 12 CES classes: Hiking, Biking, Boating, Camping, Fishing, Horseback Riding, Hunting, Shelling, Surfing, Swimming, Wildlife Viewing, Landscape Aesthetics. We define each class as follow. Hiking: the activity of going for long walks, especially in the country or woods. Biking: the activity of riding a bicycle. Boating: rowing or sailing in boats as a sport or form of recreation. Camping: the activity of spending a vacation living in a camp, tent, or camper. Fishing: the activity of catching fish. Horse riding: the sport or activity of riding horses. Hunting: the activity of hunting wild animals. Shelling: activity of collecting seashells. Surfing: the sport or pastime of riding a wave towards the shore while standing or lying on a surfboard. Swimming: swimming. Wildlife viewing: the act of observing wild animals and plants in their natural environment. Landscape aesthetics: the enjoyment and pleasure felt through the observation of environmental scenery. Now determine if an image contains one of the above 12 CES activities. If yes, tell me which class it belongs to. Only give me short answers, no need to provide the reasoning."
question = "Cultural ecosystem services (CES) are intangible, non-material benefits derived from nature through direct human outdoor interactions with the natural environment. In this study, we are interested in detecting 12 CES classes: Hiking, Biking, Boating, Camping, Fishing, Horseback Riding, Hunting, Shelling, Surfing, Swimming, Wildlife Viewing, Landscape Aesthetics. We define each class as follow. Hiking: the activity of going for long walks, especially in the country or woods. Biking: the activity of riding a bicycle. Boating: rowing or sailing in boats as a sport or form of recreation. Camping: the activity of spending a vacation living in a camp, tent, or camper. Fishing: the activity of catching fish. Horse riding: the sport or activity of riding horses. Hunting: the activity of hunting wild animals. Shelling: activity of collecting seashells. Surfing: the sport or pastime of riding a wave towards the shore while standing or lying on a surfboard. Swimming: swimming. Wildlife viewing: the act of observing wild animals and plants in their natural environment. Landscape aesthetics: the enjoyment and pleasure felt through the observation of environmental scenery. Now determine if an image contains one of the above 12 CES activities. If yes, tell me which class it belongs to. Only give me short answers, no need to provide the reasoning."
#question = "Cultural ecosystem services (CES) are intangible, non-material benefits derived from nature through direct human outdoor interactions with the natural environment. In this study, we are interested in detecting 12 CES classes: Hiking, Biking, Boating, Camping, Fishing, Horseback Riding, Hunting, Shelling, Surfing, Swimming, Wildlife Viewing, Landscape Aesthetics. We define each class as follow. Hiking: the activity of going for long walks, especially in the country or woods. Biking: the activity of riding a bicycle. Boating: rowing or sailing in boats as a sport or form of recreation. Camping: the activity of spending a vacation living in a camp, tent, or camper. Fishing: the activity of catching fish. Horse riding: the sport or activity of riding horses. Hunting: the activity of hunting wild animals. Shelling: activity of collecting seashells. Surfing: the sport or pastime of riding a wave towards the shore while standing or lying on a surfboard. Swimming: swimming. Wildlife viewing: the act of observing wild animals and plants in their natural environment. Landscape aesthetics: the enjoyment and pleasure felt through the observation of environmental scenery. Now determine if an image contains one of the above 12 CES activities. If yes, tell me which class it belongs to. If not belog 12 CES activities, reply No CES. Only give me short answers for which class, no need to provide the reasoning."
#question = "Do you see Landscape Aesthetics (the enjoyment and pleasure felt through the observation of environmental scenery)?"
#Hiking, Biking, Boating, Camping, Fishing, Horseback Riding, Hunting, Shelling, Surfing, Swimming, Wildlife Viewing, Landscape Aesthetics?"

#question = "Do you see people is biking defiend this is a person riding a bicycle, a bike, or biking equipment off-road and over rough terrain and mountain trails?"

#image_path = "./832_49357079873_actual_Boating.png"
#image_path = "./5241_27602172055_actual_No_CES.png"
#image_path = "./CES_Images//5024_48100184206_actual_No_CES.png"
image_path1 = "./CES_Images//5059_35451390872_actual_No_CES.png"
image_path2 = "./CES_Images//5027_12179796394_actual_No_CES.png"
image_path3 = "./CES_Images//5051_28946084657_actual_No_CES.png"
# multiple images/interleaved image-text
conversation = [
    {
        "role": "<|User|>",
        "content": f"This is image_1: {image_path1}\n"
                   f"This is image_2: {image_path2}\n"
                   f"This is image_3: {image_path3}\n Can you tell me what are in the images?",
        "images": [
            f"{image_path1}",
            f"{image_path2}",
            f"{image_path3}",
        ],
    },
    {"role": "<|Assistant|>", "content": ""}
]

conversation = [
    {
        "role": "<|User|>",
        "content": f"This is image_1: {image_path1}\n"
                   f"This is image_2: {image_path2}\n"
                   f"This is image_3: {image_path3}\n Can you tell me what are in the images?",
        "images": [
            f"{image_path1}",
            f"{image_path2}",
            f"{image_path3}",
        ],
    },
    {"role": "<|Assistant|>", "content": ""}
]


# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=pil_images,
    force_batchify=True,
    system_prompt=""
).to(vl_gpt.device)

# run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# run the model to get the response
outputs = vl_gpt.language.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=False)
print(f"{prepare_inputs['sft_format'][0]}", answer)

<|User|>: This is image_1: ./CES_Images//5059_35451390872_actual_No_CES.png
This is image_2: ./CES_Images//5027_12179796394_actual_No_CES.png
This is image_3: ./CES_Images//5051_28946084657_actual_No_CES.png
 Can you tell me what are in the images?

<|Assistant|>: The images provided appear to be placeholder images used in the development of a website or application. These images are typically used to indicate where actual content will be placed once the final design is ready. They are often referred to as "actual" images because they represent the final state of the content that will be displayed on the website or application.

In this case, the images are named with a sequence number (e.g., "5059_35451390872_actual_No_CES.png") and are located in the "CES_Images" directory. The "actual" label suggests that these images are intended to be used as a reference or template for the final design of the website or application.<｜end▁of▁sentence｜>
